# Quantum Nodes

import Pennylane and the numpy version from pennylane

In [7]:
import pennylane as qml
from pennylane import numpy as np

Define a qnode without parameters

In [4]:
def quantum_function1():
    qml.Hadamard(wires=0)
    return qml.expval(qml.PauliZ(0))

dev = qml.device('default.qubit', wires=1, shots=100, analytic=False)

circ = qml.QNode(quantum_function1, dev)

run the qnode

In [5]:
circ()

-0.14

Define a parametrized quantum circuit and define a qnode with it

In [9]:
def quantum_function2(x):
    qml.RX(x, wires=0)
    return qml.expval(qml.PauliZ(0))

circ = qml.QNode(quantum_function2, dev)

Run the qnode for an arbitrary value of the parameter. We choose here $\pi/2$

In [10]:
circ(np.pi)

-1.0

Add more parameters to the quantum function. We can do this by either using separate parameters or by using a parameter list.

In [12]:
def quantum_function2(x, y):
    qml.RX(x, wires=0)
    qml.RX(y, wires=1)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(1))

dev = qml.device('default.qubit', wires=2, shots=100, analytic=False)
circ = qml.QNode(quantum_function2, dev)
circ(0.0, 0.0)

1.0

In [56]:
def quantum_function2(params):
    qml.RX(params[0], wires=0)
    qml.RX(params[1], wires=1)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(1))

dev = qml.device('default.qubit', wires=2, shots=100, analytic=False)
circ = qml.QNode(quantum_function2, dev)
circ([0.0, 0.0])

1.0

# Automatic Differentiation

Take the gradient of a sinus function.

In [15]:
def f(x):
    return np.sin(x)
g = qml.grad(f)

Evaluate the gradient at 0.0

In [16]:
g(0.0)

(array(1.),)

Define a qnode and take the gradient of it and evaluate it at $\pi/2$

In [19]:
dev = qml.device("default.qubit", wires=1)
@qml.qnode(dev)
def q_f(x):
    qml.RX(x, wires=0)
    return qml.expval(qml.PauliZ(0))
q_g = qml.grad(q_f)
q_g(np.pi/2)

(array(-1.),)

Combine the qnode and the sinus function to a hybrid function and take the gardient. Evaluate it at $\pi/2$.

In [24]:
def cqh_f(x):
    out = np.sin(x)
    out = q_f(out)
    return out**2

cqh_g = qml.grad(cqh_f)
cqh_g(np.pi/2)

(array(-5.56784092e-17),)

## Draw Circuit

Draw a quantum circuit

In [25]:
dev = qml.device("default.qubit", wires=2)
@qml.qnode(dev)
def circuit(x,y):
    qml.RX(x, wires=0)
    qml.RX(y, wires=1)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(1))

circuit(np.pi, 0.0)
print(circuit.draw())

 0: ──RX(3.142)──╭C──┤     
 1: ──RX(0.0)────╰X──┤ ⟨Z⟩ 



# *args and **kwargs

Use once positional arguments explicitely and once *args.

In [26]:
def add1(x,y,z):
    return x+y+z

def add2(*num):
    result = 0
    for n in num:
        result += n
    return result

add2(1,2,3,4,5), add1(1,2,3)

(15, 6)

Use one keyword arguments and once **kwargs

In [29]:
def talk1(word1="Hello ", word2="World!"):
    return word1 + word2

def talk2(**words):
    result = ""
    for word in words.values():
        result += word
    return result

talk1(), talk2(word1 = "Hello ", word2 = "World ", word3="in ", word4="Python!")

('Hello World!', 'Hello World in Python!')

## Quantum circuits with arsg and kwargs

Use positional arguments for parameters and keyword arguments for data inputs

In [31]:
def quantum_function(theta1, theta2, x1=0, x2=0):
    qml.RX(x1, wires=0)
    qml.RY(theta1, wires=0)
    qml.RX(x2, wires=1)
    qml.RY(theta2, wires=1)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(1))
dev = qml.device('default.qubit', wires=2)
circ = qml.QNode(quantum_function, dev)  

g = qml.grad(circ)

g(0.0, 1.0)

(array(0.), array(-0.84147098))

# Multiple Quantum Nodes

Combine two quantum nodes

In [32]:
dev1 = qml.device("default.qubit", wires=1)
dev2 = qml.device("default.qubit", wires=1)
@qml.qnode(dev1)
def q_f1(theta):
    qml.RX(theta, wires=0)
    return qml.expval(qml.PauliZ(0))
@qml.qnode(dev2)
def q_f2(theta):
    qml.RX(theta, wires=0)
    return qml.expval(qml.PauliZ(0))

def f(theta):
    out = q_f1(theta)
    out = q_f2(out)
    return out**2
    
q_g = qml.grad(f)
q_g(np.pi*0.25)

(array(0.698456),)

# Optimization

Define a circuit and a cost function

In [36]:
dev = qml.device("default.qubit", wires=1)
@qml.qnode(dev)
def circuit(params, x=np.pi/2):
    qml.RX(x, wires=0)
    qml.RX(params[0], wires=0)
    return qml.expval(qml.PauliZ(0))

In [38]:
def cost(theta, x = np.pi/2):
    return circuit(theta, x=x)

### Fixed x

Optimize the circuit above for the default input $x=\pi/2$.

In [40]:
eta = 0.4
opt = qml.GradientDescentOptimizer(stepsize=eta)
steps = 100
params = np.random.rand(1)

for i in range(steps):
    params = opt.step(cost, params)
    
cost(params)

-1.0

### Flexible x

Optimize the above circuit for a different $x$.

In [42]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
params = np.random.rand(1)
x = 0.0

for i in range(steps):
    # update the circuit parameters
    params = opt.step(lambda theta : cost(theta, x=x), params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params, x=x)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.7004959
Cost after step    10: -0.8218928
Cost after step    15: -0.9987388
Cost after step    20: -0.9999924
Cost after step    25: -1.0000000
Cost after step    30: -1.0000000
Cost after step    35: -1.0000000
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [3.14159265]


# Embedding

Use the Amplitude Embedding template

In [44]:
number_of_wires = 2
dev = qml.device('default.qubit', wires=number_of_wires)
x = np.random.rand(2**number_of_wires)

@qml.qnode(dev)
def circuit(x=None):
    qml.templates.AmplitudeEmbedding(features=x, wires=[0,1], normalize=True)
    return qml.expval(qml.PauliZ(0))

circuit(x=x)

0.028324051382600968

## Broadcast

Use the broadcast template to apply a Pauli-X matrix on both wires 0 and 1.

In [45]:
@qml.qnode(dev)
def circuit(state=None):
    qml.broadcast(unitary=qml.PauliX, wires=[0,1], pattern="single")
    return [qml.expval(qml.PauliZ(i)) for i in range(number_of_wires)]

circuit()

array([-1., -1.])

Use broadcast to apply CNOT on consecutive wires.

In [49]:
pattern = [[0,1], [1,2], [0,2]]
dev = qml.device('default.qubit', wires=3)
@qml.qnode(dev)
def circuit():
    qml.broadcast(unitary=qml.CNOT, pattern=pattern, wires=[0,1,2])
    return qml.expval(qml.PauliZ(0))

circuit()

1.0

## Custom Template

Use a custom templates to make it invertible.

In [53]:
dev = qml.device('default.qubit', wires=2)

@qml.template
def Double_RX():
    qml.RX(0.5, wires=0)
    qml.RX(0.8, wires=1)
    
@qml.qnode(dev)
def circuit(state=None):
    Double_RX()
    qml.inv(Double_RX())
    return [qml.expval(qml.PauliZ(i)) for i in range(number_of_wires)]

circuit()

array([1., 1.])

Draw the circuit

In [54]:
print(circuit.draw())

 0: ──RX(0.5)──RX(0.5)⁻¹──┤ ⟨Z⟩ 
 1: ──RX(0.8)──RX(0.8)⁻¹──┤ ⟨Z⟩ 



## parameter initialization

Use the parameter initialization module to initialize the BasicEntangler template

In [55]:
n_wires = 3
n_layers = 5
dev = qml.device('default.qubit', wires=n_wires)

weights = qml.init.basic_entangler_layers_uniform(n_layers, n_wires)

@qml.qnode(dev)
def circuit(weights):
    qml.templates.BasicEntanglerLayers(weights=weights, wires=range(n_wires))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_wires)]

circuit(weights)

weights.shape

(5, 3)

## Integration other deep learning libraries

### Autograd

Use autograd for the circuit optimization

In [56]:
dev = qml.device("default.qubit", wires=1)
eta = 0.4
steps = 100

def circuit(params, x=np.pi/2):
    qml.RX(x, wires=0)
    qml.RX(params[0], wires=0)
    return qml.expval(qml.PauliZ(0))

In [58]:
def cost(theta, circ = None, x = np.pi/2):
    return circ(theta, x=x)

In [61]:
qcirc_ag = qml.QNode(circuit, dev)

opt = qml.GradientDescentOptimizer(stepsize=eta)
params = np.random.rand(1)

for i in range(steps):
    params = opt.step(lambda theta : cost(theta, circ=qcirc_ag, x=np.pi/2), params)
    
qcirc_ag(params)

-1.0

### Tensorflow

In [64]:
import tensorflow as tf

In [66]:
qcirc_tf = qml.QNode(circuit, dev, interface="tf")

params = tf.Variable(np.random.rand(1), dtype=tf.float64)

opt = tf.keras.optimizers.SGD(learning_rate=eta)

for i in range(steps):
    with tf.GradientTape() as tape:
        loss = cost(params, circ=qcirc_tf, x=np.pi/2)

    gradients = tape.gradient(loss, [params])
    opt.apply_gradients(zip(gradients, [params]))
    
qcirc_tf(params)

<tf.Tensor: id=3146, shape=(), dtype=float64, numpy=-1.0>

### Pytorch

In [67]:
import torch

In [69]:
qcirc_pt = qml.QNode(circuit, dev, interface="torch")

params = torch.tensor(np.random.rand(1), dtype=torch.float64, requires_grad=True)

opt = torch.optim.SGD([params], lr = eta)

for i in range(steps):
    loss = cost(params, circ=qcirc_pt, x=np.pi/2)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
qcirc_pt(params)

tensor(-1., dtype=torch.float64, grad_fn=<_TorchQNodeBackward>)

## Hybrid

Define a hybrid quantum classical function and optimize it in torch

In [72]:
np.random.seed(1)
qcirc_pt = qml.QNode(circuit, dev, interface="torch")
params = torch.tensor(np.random.rand(1), dtype=torch.float64, requires_grad=True)

steps = 100

def cost_hybrid_pt(theta, x = np.pi/2):
    out = torch.sin(theta)
    out = qcirc_pt(out, x=x)
    return torch.tanh(out)

opt = torch.optim.SGD([params], lr = eta)

for i in range(steps):
    loss = cost_hybrid_pt(params, x=np.pi/2)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
cost_hybrid_pt(params)

tensor(-0.6866, dtype=torch.float64, grad_fn=<TanhBackward>)

Use the same function and optimize it in tensorflow

In [74]:
np.random.seed(1)
qcirc_tf = qml.QNode(circuit, dev, interface="tf")
params = tf.Variable(np.random.rand(1), dtype=tf.float64)
opt = tf.keras.optimizers.SGD(learning_rate=eta)

def cost_hybrid_tf(theta, x = np.pi/2):
    out = tf.sin(theta)
    out = qcirc_tf(out, x=x)
    return tf.tanh(out)

for i in range(steps):
    with tf.GradientTape() as tape:
        loss = cost_hybrid_tf(params, x=np.pi/2)

    gradients = tape.gradient(loss, [params])
    opt.apply_gradients(zip(gradients, [params]))
    
cost_hybrid_tf(params)

<tf.Tensor: id=6496, shape=(), dtype=float64, numpy=-0.6865874069965548>

## Torch Layers

Take a qnode and convert it into a torch layer. This way you can add new torch layers to pre and post-process the qnodes input and output.

In [80]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights0, weights1, weights2):
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.RX(weights0, wires=0)
    qml.RX(weights1, wires=1)
    qml.templates.StronglyEntanglingLayers(weights2, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

weight_shapes = {"weights0": 1, "weights1": 1, "weights2": (3, n_qubits, 3)}

qlayer_torch = qml.qnn.TorchLayer(qnode, weight_shapes)
clayer1 = torch.nn.Linear(2, 2)
clayer2 = torch.nn.Linear(2, 2)
model_torch = torch.nn.Sequential(clayer1, qlayer_torch, clayer2)

opt = torch.optim.SGD(model_torch.parameters(), lr=0.5)
loss = torch.nn.L1Loss()

In [81]:
x = torch.tensor(np.random.rand(2), dtype=torch.float)
model_torch(x)

tensor([-0.3362, -0.3828], grad_fn=<AddBackward0>)

## Keras Layer

Do the same for Tensorflow Keras Layers.

In [82]:
qlayer_keras = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=2)
clayer1 = tf.keras.layers.Dense(2)
clayer2 = tf.keras.layers.Dense(2)
model_tf = tf.keras.models.Sequential([clayer1, qlayer_keras, clayer2])

x = tf.constant([np.random.rand(2)], dtype=tf.float32)
model_tf(x)

<tf.Tensor: id=6712, shape=(1, 2), dtype=float32, numpy=array([[-0.5499054 ,  0.04294402]], dtype=float32)>